<a href="https://colab.research.google.com/github/kgh1997/Dacon_Stock_Prediction_Algorithm/blob/main/Derived_Variable.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pykrx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.2/52.2 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 21.6 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import random
import os

from tqdm import tqdm
#from statsmodels.tsa.arima.model import ARIMA
from pykrx import stock
import re

In [ ]:
np.random.seed(42)

In [ ]:
train = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/datasets/train.csv")

In [ ]:
train

,일자,종목코드,종목명,거래량,시가,고가,저가,종가
0,20210601,A060310,3S,166690,2890,2970,2885,2920
1,20210601,A095570,AJ네트웍스,63836,5860,5940,5750,5780
2,20210601,A006840,AK홀딩스,103691,35500,35600,34150,34400
3,20210601,A054620,APS,462544,14600,14950,13800,14950
4,20210601,A265520,AP시스템,131987,29150,29150,28800,29050
...,...,...,...,...,...,...,...,...
987995,20230530,A189980,흥국에프엔비,272284,3005,3035,2955,2980
987996,20230530,A000540,흥국화재,50218,3250,3255,3195,3215
987997,20230530,A003280,흥아해운,130664,1344,1395,1340,1370
987998,20230530,A037440,희림,141932,9170,9260,9170,9200


In [ ]:
train.info() # 가격 변동성 ,기술지표(RSI, MACD, 볼린저 밴드 등), 지연된 변수, 시간 특징, 시장 지수

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 988000 entries, 0 to 987999
Data columns (total 8 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   일자      988000 non-null  int64 
 1   종목코드    988000 non-null  object
 2   종목명     988000 non-null  object
 3   거래량     988000 non-null  int64 
 4   시가      988000 non-null  int64 
 5   고가      988000 non-null  int64 
 6   저가      988000 non-null  int64 
 7   종가      988000 non-null  int64 
dtypes: int64(6), object(2)
memory usage: 60.3+ MB


### 파생변수

In [ ]:
# train 데이터에서 종목코드 열의 고유한 값들을 추출
unique_codes = train['종목코드'].unique()
# train 데이터에서 종목명 열의 고유한 값들을 추출
code_names = train["종목명"].unique()
# 빈 데이터프레임 stock_df를 생성
stock_df = pd.DataFrame()
# 각 종목코드와 종목명에 대해 반복문
for code,name in zip(tqdm(unique_codes),(code_names)):
    # fromdate부터 todate까지의 시장 OHLCV 데이터를 가져오기
    df = stock.get_market_ohlcv_by_date(fromdate="20210401", todate="20210530", ticker=code[1:]).reset_index()
    df["종목코드"] = code
    df["종목명"] = name
    # df에서 원하는 컬럼 쿼리로 갖고오기
    df = df[["날짜","종목코드","종목명","거래량","시가","고가","저가","종가"]]
    # stock_df에 df를 추가합니다.
    stock_df = stock_df.append(df)
# stock_df를 날짜 기준으로 정렬하고 인덱스를 재설정합니다.
stock_df = stock_df.sort_values("날짜").reset_index(drop=True)

  0%|          | 0/2000 [00:03<?, ?it/s]


KeyboardInterrupt: ignored

In [ ]:
import datetime
stock_df['날짜'] = stock_df['날짜'].apply(lambda x : x.strftime('%Y%m%d')) # 날짜 컬럼을 yymmdd형식으로 바꾸기
stock_df.rename({"날짜":"일자"},axis=1,inplace=True) # 날짜를 일자로 변경
stock_df["일자"]= stock_df["일자"].astype(int) # 일자를 숫자로 변경

In [ ]:
# stock_df와 train을 합치기
train = pd.concat([stock_df, train], axis=0)

In [ ]:
## 일일 수익률, VWAP(거래량 가중 평균 가격으로, 특정 기간 동안의 평균 가격),
## ATR(주식의 변동성을 측정하는 지표), RSI(주식의 상대적 강도를 측정하는 지표), CCI(현재의 주가가 이동평균에서 상하 얼마나 떨어져 있는지),

### 일일 수익률

In [ ]:
# 일일 수익률
train['일일 수익률'] = train.groupby("종목코드")['종가'].pct_change() # 종목코드 별로 그룹바이 한 다음 종가를 이용해서 수익률 계산
train["일일 수익률"].fillna(0, inplace=True)

### 누적 수익률

In [ ]:
result2 = pd.DataFrame()
# 각 종목코드에 대해서 모델 학습 및 추론 반복
for code in tqdm(unique_codes):
    df = train[train['종목코드'] == code]
    # df의 '일일 수익률' 열의 누적 합계를 계산하여 '누적 수익률' 열에 저장합니다.
    df["누적 수익률"] = df["일일 수익률"].cumsum()
    # 결측치를 0으로 채우기
    df["누적 수익률"] = df["누적 수익률"].fillna(0)
    # result2에 추가
    result2 = result2.append(df)
result2.sort_values("일자").reset_index(drop=True, inplace=True)

100%|██████████| 2000/2000 [04:04<00:00,  8.17it/s]


### ATR

In [ ]:
result5 = pd.DataFrame()
# 각 종목코드에 대해서 모델 학습 및 추론 반복
for code in tqdm(unique_codes):
    df = train[train['종목코드'] == code]
    # 가격 변동성(ATR)
    high = df['고가']
    low = df['저가']
    # 종가 컬럼의 이전 값들을 close_prev에 저장
    close_prev = df['종가'].shift(1)
    # 고가-저가, 고가-close_prev, 저가-close_prev 중 큰 값을 true_range에 저장
    true_range = np.maximum(high - low, np.abs(high - close_prev), np.abs(low - close_prev))

    # ATR 계산하기 (기간은 14일)
    window = 14
    # true_range의 14일 이동평균
    atr = true_range.rolling(window=window).mean()
    df['ATR'] = atr
    df["ATR"] = df["ATR"].fillna(0)
    result5 = result5.append(df)
result5.sort_values("일자").reset_index(drop=True, inplace=True)

100%|██████████| 2000/2000 [04:09<00:00,  8.01it/s]


In [ ]:
result5["일자"]=result5["일자"].astype(int)
result5.sort_values("일자").reset_index(drop=True, inplace=True)

### VWAP

In [ ]:
def calculate_vwap(data):
    # Volume_price: 거래량과 종가의 곱
    data['Volume_Price'] = data['거래량'] * data['종가']
    # 누적 Volume_price: 일자별로 Volume_price의 누적 합
    data['Cumulative_Volume_Price'] = data.groupby('일자')['Volume_Price'].cumsum()
    # '일자' 별로 '거래량' 열의 누적 합계를 계산
    data['Cumulative_Volume'] = data.groupby('일자')['거래량'].cumsum()
    # 'Cumulative_Volume_Price' 열을 'Cumulative_Volume' 열로 나누어 VWAP 값을 계산하고 'VWAP' 열에 저장합니다
    data['VWAP'] = data['Cumulative_Volume_Price'] / data['Cumulative_Volume']
    # 선형보간법으로 결측치 채우기
    data["VWAP"] = data["VWAP"].interpolate(method='linear')
    return data

In [ ]:
result5 = pd.DataFrame()
# 각 종목코드에 대해서 모델 학습 및 추론 반복
for code in tqdm(unique_codes):
    df = train[train['종목코드'] == code]
    # VWAP
    #vwap = ta.volume.VolumeWeightedAveragePrice(volume = df["거래량"], low = df["저가"], high = df["고가"], close = df["종가"], window=1)
    df = calculate_vwap(df)
    #df["VWAP"] = df["VWAP"].interpolate(method='linear')
    result5 = result5.append(df)
result5 = result5.sort_values("일자").reset_index(drop=True)

100%|██████████| 2000/2000 [04:24<00:00,  7.58it/s]


In [ ]:
result["일자"]=result["일자"].astype(int)
result.sort_values("일자").reset_index(drop=True, inplace=True)

### RSI

In [ ]:
def calculate_rsi(data, window=14):
    # 종가열의 차이를 계산
    price_diff = data['종가'].diff()
    # price_diff에서 음수값을 0으로 변경하여 gain 변수에 저장
    gain = price_diff.mask(price_diff < 0, 0)
    # price_diff에서 양수값을 0으로 변경하여 loss 변수에 저장합니다.
    loss = -price_diff.mask(price_diff > 0, 0)
    # gain의 이동평균 계산
    avg_gain = gain.rolling(window).mean()
    # loss의 이동평균 계산
    avg_loss = loss.rolling(window).mean()
    # avg_gain을 avg_loss로 나누어 rs 값을 계산
    rs = avg_gain / avg_loss
    # rs 값을 이용하여 rsi 값을 계산 ( 100 - 100 / ( 1+ rs))
    rsi = 100 - (100 / (1 + rs))

    return rsi

In [ ]:
result3 = pd.DataFrame()
# 각 종목코드에 대해서 모델 학습 및 추론 반복
for code in tqdm(unique_codes):
    df = train[train['종목코드'] == code]
    # RSI
    #rsi = ta.momentum.RSIIndicator(close = df["종가"], window=14)
    df["RSI"] = calculate_rsi(df, window=14)
    df["RSI"] = df["RSI"].interpolate(method='linear')
    result3 = result3.append(df)
result3 = result3.sort_values("일자").reset_index(drop=True)

100%|██████████| 2000/2000 [04:16<00:00,  7.80it/s]


In [ ]:
result3["일자"]=result3["일자"].astype(int)
result3.sort_values("일자").reset_index(drop=True, inplace=True)

### CCI

In [ ]:
def calculate_typical_price(data):
    # 일반 가격을 계산, (고가 + 저가 + 종가) / 3
    data['일반가격'] = (data['고가'] + data['저가'] + data['종가']) / 3
    return data

def calculate_mean_deviation(data, window):
    # 평균 편차를 계산, 일반 가격 - 이동 평균(일반 가격)
    data['평균편차'] = np.abs(data['일반가격'] - data['일반가격'].rolling(window=window).mean())
    return data

def calculate_cci(data, window):
    # CCI를 계산, (일반 가격 - 이동 평균(일반 가격)) / (0.015 * 이동 평균(평균 편차))
    mean_deviation = data['평균편차'].rolling(window=window).mean()
    data['CCI'] = (data['일반가격'] - data['일반가격'].rolling(window=window).mean()) / (0.015 * mean_deviation)
    return data

In [ ]:
result4 = pd.DataFrame()
# 각 종목코드에 대해서 모델 학습 및 추론 반복
for code in tqdm(unique_codes):
    df = train[train['종목코드'] == code]
    # CCI
    #cci = ta.trend.CCIIndicator(high= df["고가"],low=df["저가"],close = df["종가"], window=14, fillna=False)
    # 일반가격 계산
    df = calculate_typical_price(df)
    # 평균편차 계산
    df = calculate_mean_deviation(df, window=14)
    # CCI 계산
    df = calculate_cci(df, window=14)
    #df["CCI"] = cci.cci()
    #df["CCI"] = df["CCI"].interpolate(method='linear')
    result4 = result4.append(df)
result4 = result4.sort_values("일자").reset_index(drop=True)

100%|██████████| 2000/2000 [04:24<00:00,  7.55it/s]


In [ ]:
#result["VWAP"] = result5["VWAP"]
#result["RSI"] = result3["RSI"]
#result["CCI"] = result4["CCI"]
result["누적 수익률"] = result2["누적 수익률"]
#result = result.sort_values("일자").reset_index(drop=True)
#result

In [ ]:
result = result[result["일자"]>=20210601].reset_index(drop=True)

In [ ]:
# 이전의 값으로 채우기
result = result.fillna(method= 'ffill')

In [ ]:
result["이슈"] = 0
result["이슈"] = result.apply(lambda x: 1 if x["일일 수익률"] >= 3 else 0, axis=1)